In [3]:
import os
import sys
import openai
__file__ = '../../'
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
sys.path.append(os.path.abspath(__file__))  # Agrega el directorio anterior al PATH

from drf import settings


In [4]:
prompt = 'Ayuda a saludar en portugues'

In [5]:
mensajes = [
    {"role": "system", "content": "Eres un asistente virtual que ayuda con preguntas de español a portugues y viceversa."},
    {"role": "user", "content": prompt},
]

In [6]:
openai.api_key = settings.API_KEY_OPENAI

respuesta = openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = mensajes,
    max_tokens = 100,
    temperature = 0,
)

In [7]:
print(respuesta.choices[0].message.content)

¡Claro! Aquí tienes algunas formas de saludar en portugués:

- Hola: Olá
- Buenos días: Bom dia
- Buenas tardes: Boa tarde
- Buenas noches: Boa noite

Espero que te sean útiles. ¿Hay algo más en lo que pueda ayudarte?


In [55]:
def obtener_completion( contexto , model = 'gpt-3.5-turbo'):
    mensaje = openai.chat.completions.create(
        model = model,
        messages = contexto,
        max_tokens = 100,
        temperature = 0,
    )
    return mensaje.choices[0].message.content

In [56]:
contexto = [
    {"role": "system", "content": "Eres un asistente virtual que ayuda con preguntas de español a portugues y viceversa."},
    {"role": "user", "content": "Ayudame a saludar en portugues"},
]

respuesta = obtener_completion(contexto)

In [57]:
print(respuesta)

¡Claro! Para saludar en portugués puedes decir "Olá" que significa "Hola" o "Bom dia" que significa "Buenos días". ¿Hay algo más en lo que pueda ayudarte?


In [58]:
# Crearemos un varaible global que contenga el contexto de la conversacion

contexto = [{"role": "assistant", "content": "Ayudame a saludar en portugues"}]

contento = contexto + [{"role": "system", "content": respuesta}]

print(contento)

[{'role': 'user', 'content': 'Ayudame a saludar en portugues'}, {'role': 'system', 'content': '¡Claro! Para saludar en portugués puedes decir "Olá" que significa "Hola" o "Bom dia" que significa "Buenos días". ¿Hay algo más en lo que pueda ayudarte?'}]


In [5]:
contexto = [
    {"role": "system", "content": "Eres un asistente virtual que ayuda con preguntas de español a portugues y viceversa."},
    {"role": "user", "content": "Ayudame a saludar en portugues"},
]

In [11]:
from groq import Groq

groq_client =Groq(
    api_key = settings.API_KEY_GROQ,
)

respuesta = groq_client.chat.completions.create(
    model = 'llama3-8b-8192',
    messages = contexto, 
    max_tokens = 300,   
    temperature = 0,
)

NameError: name 'contexto' is not defined

In [11]:
print(respuesta.choices[0].message.content)

Um prazer!

En portugués, hay varias formas de saludar dependiendo del momento del día y la región. Aquí te presento algunas opciones:

* Bom dia (bom DEE-ah): Hola, buenos días
* Boa tarde (BOH-ah TAR-deh): Hola, buenas tardes
* Boa noite (BOH-ah NOH-ee-teh): Hola, buenas noches
* Tudo bem? (TOO-doo BEM): ¿Cómo estás? (una forma informal de saludar)

Si deseas ser más formal, puedes usar:

* Olá (OH-lah): Hola
* Obrigado/Obrigada (oh-bree-GAH-doo/oh-bree-GAH-dah): Gracias (si eres hombre/mujer)

Espero que te sea útil!


In [15]:
contexto = [{
            "data": [],
            "personalidad": "sad",
        }]
print(contexto[0]['data'])

[]


In [20]:
import os
import sounddevice as sd
from scipy.io.wavfile import write
from groq import Groq

def record_audio(filename, duration=5, samplerate=44100):
    print("Recording...")
    audio = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=2, dtype='int16')
    sd.wait()  # Esperar a que termine la grabación
    write(filename, samplerate, audio)  # Guardar el archivo de audio
    print("Recording finished")

def transcribe_audio(filename, api_key):
    client = Groq(api_key=api_key)

    with open(filename, "rb") as file:
        #translations
        #transcriptions
        transcription = client.audio.transcriptions.create(
            file=(filename, file.read()),
            model="whisper-large-v3",
            prompt="Specify context or spelling",  # Opcional
            response_format="json",  # Opcional
            language="pt",  # Opcional
            temperature=0.0  # Opcional
        )
        return transcription.text

def main():
    audio_filename = "recorded_audio.wav"
    api_key = settings.API_KEY_GROQ  # Reemplaza con tu API key de Groq
    record_audio(audio_filename, duration=5)  # Grabar 5 segundos de audio
    transcription = transcribe_audio(audio_filename, api_key)
    print("Transcription:", transcription)

if __name__ == "__main__":
    main()

# pip install sounddevice scipy

Recording...
Recording finished
Transcription:  kawal e o seu nome


In [24]:
import os
import sounddevice as sd
from scipy.io.wavfile import write
from groq import Groq
from difflib import SequenceMatcher

def record_audio(filename, duration=5, samplerate=44100):
    print("Recording...")
    audio = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=2, dtype='int16')
    sd.wait()  # Esperar a que termine la grabación
    write(filename, samplerate, audio)  # Guardar el archivo de audio
    print("Recording finished")

def transcribe_audio(filename, api_key):
    client = Groq(api_key=api_key)

    with open(filename, "rb") as file:
        transcription = client.audio.transcriptions.create(
            file=(filename, file.read()),
            model="whisper-large-v3",
            prompt="",  # Opcional
            response_format="json",  # Opcional
            language="pt",  # Cambiar a portugués
            temperature=0.0  # Opcional
        )
        return transcription.text

def compare_transcription(transcription, reference_text):
    # Usar SequenceMatcher para comparar la transcripción con el texto de referencia
    matcher = SequenceMatcher(None, transcription, reference_text)
    score = matcher.ratio()  # Obtener la similitud entre los dos textos
    return score

def evaluate_pronunciation(reference_text, duration=5):
    audio_filename = "recorded_audio.wav"
    api_key = os.getenv("API_KEY_GROQ")
    if not api_key:
        raise ValueError("La variable de entorno GROQ_API_KEY no está configurada.")

    record_audio(audio_filename, duration=duration)  # Grabar el audio
    transcription = transcribe_audio(audio_filename, api_key)  # Transcribir el audio
    print(f"Transcription: {transcription}")
    
    score = compare_transcription(transcription, reference_text)  # Comparar con el texto de referencia
    print(f"Pronunciation score: {score:.2f}")

    return score

if __name__ == "__main__":
    reference_text = "Este é o texto de referência para comparar."  # Texto de referencia en portugués
    evaluate_pronunciation(reference_text, duration=5)


Recording...
Recording finished
Transcription:  O que está acontecendo? Texto de referência para comparar.
Pronunciation score: 0.71
